In [ ]:
!pip install mteb sentence-transformers transformers datasets evaluate pymongo

In [ ]:

# Import the task directly
from mteb.tasks import Banking77Classification

# Load the task
task = Banking77Classification()

# Load dataset
task.load_data()

# Now you can access the splits
train_data = task.dataset["train"]
test_data = task.dataset["test"]

# Print sample
print(train_data[0])


# ✅ STEP 1: Install dependencies


# ✅ STEP 2: Load dataset from MTEB
from mteb.tasks import Banking77Classification
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import torch

# Load the classification task
task = Banking77Classification()
task.load_data()

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_list(task.dataset["train"])
test_dataset = Dataset.from_list(task.dataset["test"])

# ✅ STEP 3: Tokenization
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# ✅ STEP 4: Load model
num_labels = len(set(train_dataset["labels"]))
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels)

# ✅ STEP 5: Metric for evaluation
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# ✅ STEP 6: Training configuration
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

# ✅ STEP 7: Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

# ✅ STEP 8: Save model & tokenizer
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


In [ ]:
from transformers import pipeline
from datasets import load_dataset

# ✅ Load model and tokenizer from saved directory
classifier = pipeline("text-classification", model="saved_model", tokenizer="saved_model", device=0)  # Use CUDA if available

# ✅ Load the original label names from Banking77
label_names = load_dataset("banking77")["train"].features["label"].names

# ✅ Example queries for prediction
test_queries = [
    "I want to change my PIN",
    "Where can I find my account balance?",
    "How do I cancel a transaction?",
    "Please block my card immediately",
    "I forgot my account password"
]

# ✅ Run predictions and map label index to intent name
for query in test_queries:
    pred = classifier(query)[0]
    label_index = int(pred["label"].replace("LABEL_", ""))
    intent_name = label_names[label_index]

    print(f"Query: {query}")
    print(f"Predicted intent: {intent_name} (Label {label_index}), confidence: {pred['score']:.2f}")
    print("------")


In [ ]:
import torch
import os

# Create directory to save
os.makedirs("saved_model", exist_ok=True)

# Save only the state_dict (weights)
torch.save(model.state_dict(), "saved_model/pytorch_model.bin")

tokenizer.save_pretrained("saved_model")


In [ ]:
# Zip the folder
!zip -r saved_model.zip saved_model/

# Download in Colab
from google.colab import files
files.download("saved_model.zip")


In [ ]:

import json

# Set save directory
SAVE_DIR = "saved_model"
os.makedirs(SAVE_DIR, exist_ok=True)

# ✅ 1. Save the model
model.save_pretrained(SAVE_DIR)

# ✅ 2. Save the tokenizer
tokenizer.save_pretrained(SAVE_DIR)

# ✅ 3. Save the intent label names as a JSON file
with open(os.path.join(SAVE_DIR, "intent_labels.json"), "w") as f:
    json.dump(label_names, f)

print(f"✅ Model, tokenizer, and intent_labels.json saved in: {SAVE_DIR}")


translation training

In [ ]:
# Run this cell first to download and cache translation model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from huggingface_hub import login
login()  # Will prompt for token
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Save locally
tokenizer.save_pretrained("nllb-600M")
model.save_pretrained("nllb-600M")
print("Model saved locally")

In [ ]:
# Run this cell once to save the translation model to Google Drive
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Set your Google Drive path
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks"
MODEL_PATH = f"{DRIVE_PATH}/nllb-600M"

# Download and save model
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Save to Google Drive
tokenizer.save_pretrained(MODEL_PATH)
model.save_pretrained(MODEL_PATH)
print(f"Saved translation model to {MODEL_PATH}")
# Method 1: Colab secrets (recommended)
# 1. Go to: https://colab.research.google.com/
# 2. Click the key icon in the left sidebar
# 3. Add a secret named "NGROK_AUTH_TOKEN" with your ngrok token



database setup

In [ ]:
# !pip install pymongo



from pymongo import MongoClient

# Connect using your new URI
uri = "mongodb+srv://<username>:<db_password>@cluster1.nykrkvq.mongodb.net/?retryWrites=true&w=majority&appName=Cluster1"  # Replace <username> and <db_password> with your actual username and password
client = MongoClient(uri)

# Choose your database and collection
db = client["travis"]  # You can name this anything (will be created automatically)
collection = db["users"]  # This is your collection (like a table)

# Sample data
user_data = {
    "name": "Rajesh",
    "email": "rajesh@example.com",
    "password": "123456"
}

# Insert user
if collection.find_one({"email": user_data["email"]}):
    print("✅ User already exists")
else:
    collection.insert_one(user_data)
    print("✅ User added successfully")



In [ ]:
from pymongo import MongoClient

# Step 1: Connect to your cluster
uri = "mongodb+srv://<username>:<db_password>@cluster1.nykrkvq.mongodb.net/?retryWrites=true&w=majority&appName=Cluster1" # Replace <username> and <db_password> with your actual username and password
client = MongoClient(uri)

# Step 2: Choose DB and collection
db = client["travis_db"]
collection = db["intent_responses"]

# Step 3: Define the intent-response dictionary
intent_responses = {
    "activate_card": "To activate your card, please follow the instructions provided with it or use our mobile app.",
    "age_limit": "The minimum age requirement to open an account is 18 years.",
    "apple_pay_or_google_pay": "Yes, we support Apple Pay and Google Pay. You can add your card through your phone’s wallet app.",
    "atm_support": "You can use your card at any ATM that supports our bank. Fees may apply for non-partner ATMs.",
    "automatic_top_up": "You can enable automatic top-up from your account settings under the 'Top-Up' section.",
    "balance_not_updated_after_bank_transfer": "Sometimes it may take a few hours for your balance to reflect. Please wait or contact support if it's delayed.",
    "balance_not_updated_after_cheque_or_cash_deposit": "Cheque or cash deposits can take up to 2–3 business days to reflect.",
    "beneficiary_not_allowed": "Please ensure the beneficiary account is valid and accepted by our system. Try again or contact support.",
    "cancel_transfer": "If the transfer is still pending, you can cancel it from your transaction history.",
    "card_about_to_expire": "We’ll automatically send a new card before your current one expires.",
    "card_acceptance": "Our card is accepted globally anywhere Visa/Mastercard is supported.",
    "card_arrival": "Your card should arrive within 5–7 business days after approval.",
    "card_delivery_estimate": "Card delivery usually takes 5–7 working days. You can track it from your account.",
    "card_linking": "To link your card, go to ‘Card Settings’ and select ‘Link to Account’ or ‘Wallet’.",
    "card_payment_fee_charged": "Some merchants may apply a fee. Check the transaction details or contact us for help.",
    "card_payment_not_recognised": "If you don’t recognize the payment, you can freeze your card and report the transaction.",
    "card_payment_wrong_exchange_rate": "Exchange rates are set by the card network. Rate fluctuations may occur.",
    "card_swallowed": "Please visit the branch or call support immediately to retrieve your card.",
    "cash_withdrawal_charge": "We don’t charge for ATM withdrawals, but third-party ATMs may apply a fee.",
    "cash_withdrawal_not_recognised": "If you didn’t make the withdrawal, freeze your card and report the issue.",
    "change_pin": "You can change your card PIN from the mobile app or at any supported ATM.",
    "compromised_card": "Please freeze your card immediately and order a replacement.",
    "contactless_not_working": "Make sure contactless is enabled. If issues continue, request a card replacement.",
    "country_support": "We currently operate in select countries. Please check our supported countries list on the website.",
    "declined_card_payment": "Card payments can be declined due to limits or security checks. Check your balance or try again.",
    "declined_cash_withdrawal": "This may be due to ATM limits or account issues. Try another machine or contact support.",
    "declined_transfer": "Transfers may be declined due to limits or incorrect details. Please verify and try again.",
    "direct_debit_payment_not_recognised": "If you don’t recognize the debit, cancel it and contact support for help.",
    "disposable_card_limits": "Disposable cards have a limited spending cap. Check your app for details.",
    "edit_personal_details": "To update your details, go to your profile and select ‘Edit Information’.",
    "exchange_charge": "Currency exchanges include a small fee. You can view the rate before confirming.",
    "exchange_rate": "Our rates are competitive and updated in real-time. You can check them in the app.",
    "exchange_via_app": "Yes, you can exchange currencies within the app under the ‘Currency Exchange’ section.",
    "extra_charge_on_statement": "Check your transaction details for merchant-added fees or currency conversion.",
    "failed_transfer": "Transfers may fail due to network or verification issues. Retry or contact support.",
    "fiat_currency_support": "We support multiple fiat currencies. Check the app for the full list.",
    "get_disposable_virtual_card": "Go to Cards > Add Disposable Card to get a virtual one instantly.",
    "get_physical_card": "You can request a physical card from the app. Delivery takes 5–7 business days.",
    "getting_spare_card": "You can order a spare card in case of loss or damage from your card settings.",
    "getting_virtual_card": "Virtual cards can be created instantly from the app under ‘Cards’ section.",
    "lost_or_stolen_card": "Please freeze the card immediately and request a replacement.",
    "lost_password": "Use the ‘Forgot Password’ option on the login screen to reset your password.",
    "order_physical_card": "Tap ‘Order Card’ in the app and fill in your delivery address.",
    "passcode_forgotten": "Tap ‘Forgot Passcode’ and follow the steps to reset it securely.",
    "pending_card_payment": "Some payments may show as pending until the merchant finalizes it.",
    "pending_cash_withdrawal": "This can take a few minutes to process. Contact support if it persists.",
    "pending_top_up": "Top-ups may take a few minutes. Please wait and refresh your balance.",
    "pin_blocked": "If your PIN is blocked, you can reset it from the app or at an ATM.",
    "reactivated_card": "You can reactivate your card from the app if it was deactivated recently.",
    "receiving_money": "Share your account number or payment link to receive money securely.",
    "request_refund": "To request a refund, select the transaction and tap ‘Report or Refund’.",
    "reverted_card_payment": "Sometimes merchants cancel transactions, and the amount is returned to your balance.",
    "supported_cards_and_currencies": "We support most major cards and a wide range of currencies.",
    "terminate_account": "To close your account, go to Settings > Account > Terminate Account.",
    "top_up_by_bank_transfer_charge": "Bank top-ups are free from our side, but your bank may charge a fee.",
    "top_up_by_card_charge": "Card top-ups may include a processing fee shown before confirmation.",
    "top_up_failed": "Check your card or bank account details and try the top-up again.",
    "top_up_limits": "Top-ups may have daily or monthly limits. Check your app for the current limit.",
    "top_up_reverted": "If a top-up was reverted, the money should return to your source account shortly.",
    "topping_up_by_card": "To top-up using a card, go to the app’s top-up section and select ‘Card’ as method.",
    "transaction_charged_twice": "If you were double-charged, report the transaction and we’ll investigate.",
    "transfer_fee_charged": "Some transfers may include a fee, depending on the method or currency.",
    "transfer_into_account": "You can transfer funds into your account via bank transfer or another linked account.",
    "transfer_not_received_by_recipient": "Ensure the recipient’s details were correct. It can take up to 24 hours.",
    "transfer_timing": "Transfers are usually instant, but some banks may take up to 1–2 business days.",
    "unable_to_verify_identity": "Please upload clear and valid ID documents. Contact support if issues persist.",
    "verify_my_identity": "Go to Settings > Identity Verification and follow the steps with valid ID proof.",
    "verify_source_of_funds": "We may ask for proof of income or transaction history for verification.",
    "verify_top_up": "Sometimes we require extra verification for top-ups. Follow the instructions shown.",
    "virtual_card_not_working": "Try removing and re-adding the card. If the issue persists, request a new one.",
    "visa_or_mastercard": "We issue both Visa and Mastercard depending on your region.",
    "why_verify_identity": "Identity verification helps keep your account secure and prevent fraud.",
    "wrong_amount_of_cash_received": "Please check the ATM receipt and report the incorrect amount.",
    "wrong_exchange_rate_for_cash_withdrawal": "ATM providers may use their own exchange rates. Check before confirming."
}

# Step 4: Convert dict to list of documents
documents = [{"intent": intent, "response": response} for intent, response in intent_responses.items()]

# Step 5: Insert into MongoDB (optional: clear old first)
collection.delete_many({})  # optional: clear old data
collection.insert_many(documents)

print("✅ All intent-responses inserted into MongoDB!")
